In [1]:
import os
import sys
import json
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2017,1)

In [2]:
def get_neighbors(maze, point, rules={}):
    """
    Function to get neighbors of a point on a map or maze
    This function assumes screen coordinates.  If using another coordinate system,
    please update. Maybe a rule flag to specify?

    Notes: see 2023.21 for infinite complex example

    Args:
        maze: list_x(list_y()) or dict(tuple(x,y) or dict(complex())) 
        point: tuple(x,y) or complex() # should match maze, or things may break
        rules: dict{} , example:
            rules = {
                "type": "bounded", # or infinite
                "invalid": "#",
                "coordinate_system": "screen" # or matrix, or cartesian, others noted below, 
                    are not yet supported
            }
    Returns:
        neighbors: list(tuple(x,y)) or list(complex())

    Notes:
        tuple to complex:
            complex(my_tuple)
        complex to tuple:
            tuple(my_complex.real, my_complex.imag)
        Coordinate System	X Increases	Y Increases	Common Use
        Screen Coordinates	To the right	Down	Computer graphics, UI, web design
        Matrix Coordinates	To the right (cols)	Down (rows)	Spreadsheets, grid-based systems
        Cartesian Coordinates	To the right	Up	Mathematics, physics, engineering
        Polar Coordinates	N/A (radius and angle)	N/A	Navigation, physics, engineering
        Geographic Coordinates	N/A (longitude)	N/A (latitude)	Geography, GPS
        Isometric Coordinates	120-degree intervals	120-degree intervals	Video games,
            CAD, technical drawing
    """
    X=0
    Y=1
    # I think I'm getting technical here, but this may matter when we go to apply rules
    # as I typically provide matrix coordinates as (row, col)
    if rules.get('coordinate_system', 'screen') ==  'matrix':
        X=1
        Y=0
    # define offsets
    # change aoc.self when we move this into the class
    offsets = my_aoc.get_neighbor_offsets()

    # empty list of neighbors
    neighbors = []
    if isinstance(point, complex):
        for offset in offsets["complex"]:
            neighbors.append(point + offset)
    else:
        for offset in offsets["tuple"]:
            neighbors.append(tuple([point[X] - offset[X], point[Y] - offset[Y]]))
            
    # process rule type:bounded
    if rules.get("type", "bounded") == "bounded":
        min, max = get_maze_size(maze)
        if isinstance(maze, dict):
            for neighbor in neighbors:
                if not neighbor in maze:
                    neighbors.pop(neighbor)
        else:
            valid_neighbors = []
            for neighbor in neighbors:
                if min[X] <= neighbor[X] <= max[X] and min[Y] <= neighbor[Y] <= max[Y]:
                    valid_neighbors.add(neighbor)
            neighbors = valid_neighbors
            
    return neighbors

def get_maze_size(maze):
    """
    Function to get min(X,Y), max(X,Y) for maze
    """
    X=0
    Y=1
    if isinstance(maze, list):
        # list of list, return 0 to length
        min = tuple([0, 0])
        max = tuple([len(maze), len(maze[0])])
        return min, max
    if not isinstance(maze, dict):
        print(f"get_maze_size no rule to handle {type(maze)}")
        sys.exit()
    # complex or tuple?
    min = [float('infinity')]*2
    max = [float('infinity')*-1]*2
    is_complex = isinstance(list(maze.keys())[0], complex)
    for key in maze.keys():
        print(key,min,max)
        if is_complex:
            if key.real < min[X]:
                min[X] = int(key.real)
            if key.real > max[X]:
                max[X] = int(key.real)
            if key.imag < min[Y]:
                min[Y] = int(key.imag)
            if key.imag > max[Y]:
                max[Y] = int(key.imag)
        else:
            if key[X] < min[X]:
                min[X] = key[X]
            if key[X] > max[X]:
                max[X] = key[X]
            if key[Y] < min[Y]:
                min[Y] = key[Y]
            if key[Y] > max[Y]:
                max[Y] = key[Y]
    return min, max

            
        
    

In [3]:
maze={
    complex(0,0): 'a',
    complex(0,1): 'a',
    complex(1,0): 'a',
    complex(1,1): '#',
}
point=complex(0,0)
neighbors = my_aoc.get_neighbors(maze, point, invalid="#")
print(neighbors)

[1j, (1+0j)]


In [4]:
my_tuple = (1,1)
my_complex =complex(*my_tuple)
my_complex

(1+1j)

In [5]:
my_inf =  float('infinity')
print(my_inf)
my_inf =  float('infinity') * -1
print(my_inf)


inf
-inf


In [6]:
min = [float('infinity')]*2
min

[inf, inf]

In [74]:
def reverse_captcha(input_string, part=1):
    idx_offset = 1
    int_list = [int(char) for char in input_string]
    int_list_length = len(int_list)
    if part == 2:
        idx_offset = int_list_length // 2
    total = 0
    for idx, num in enumerate(int_list):
        num2 = int_list[(idx + idx_offset) % (int_list_length)]
        if num == num2:
            total += num
    return total
    

In [78]:
reverse_captcha('57276274387944537823652626177853384411146325384494935924454336611953119173638191671326254832624841593421667683474349154668177743437745965461678636631863541462893547616877914914662358836365421198516263335926544716331814125295712581158399321372683742773423626286669759415959391374744214595682795818615532673877868424196926497731144319736445141728123322962547288572434564178492753681842244888368542423832228211172842456231275738182764232265933625119312598161192193214898949267765417468348935134618964683127194391796165368145548814473129857697989322621368744725685183346825333247866734735894493395218781464346951777873929898961358796274889826894529599645442657423438562423853247543621565468819799931598754753467593832328147439341586125262733737128386961596394728159719292787597426898945198788211417854662948358422729471312456437778978749753927251431677533575752312447488337156956217451965643454445329758327129966657189332824969141448538681979632611199385896965946849725421978137753366252459914913637858783146735469758716752765718189175583956476935185985918536318424248425426398158278111751711911227818826766177996223718837428972784328925743869885232266127727865267881592395643836999244218345184474613129823933659422223685422732186536199153988717455568523781673393698356967355875123554797755491181791593156433735591529495984256519631187849654633243225118132152549712643273819314433877592644693826861523243946998615722951182474773173215527598949553185313259992227879964482121769617218685394776778423378182462422788277997523913176326468957342296368178321958626168785578977414537368686438348124283789748775163821457641135163495649331144436157836647912852483177542224864952271874645274572426458614384917923623627532487625396914111582754953944965462576624728896917137599778828769958626788685374749661741223741834844643725486925886933118382649581481351844943368484853956759877215252766294896496444835264357169642341291412768946589781812493421379575569593678354241223363739129813633236996588711791919421574583924743119867622229659211793468744163297478952475933163259769578345894367855534294493613767564497137369969315192443795512585', 2)

1080